### LangGraph getting started examples
DBR ML 16.1

In [0]:
%pip install langgraph databricks_langchain
%restart_python

In [0]:
from typing import TypedDict, Dict, Sequence, List, Annotated, Any
from operator import add
from langgraph.graph import StateGraph, END, START, MessageGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage
from IPython.display import Image
from databricks_langchain import ChatDatabricks

In [0]:
model = ChatDatabricks(endpoint='databricks-mixtral-8x7b-instruct')

### A simple example. 
Two graph nodes in a sequence

In [0]:
class SimpleState(TypedDict):
  a: int
  b: int
  added: int
  subtracted: int


def addition(state: SimpleState) -> Dict[str, int]:
  a = state['a']
  b = state['b']
  added = a + b
  return {"added": added}


def subtraction(state: SimpleState) -> Dict[str, int]:
  a = state['a']
  b = state['b']
  added = state['added']
  # Will return 0
  subtracted = added - a - b
  return {"subtracted": subtracted}

In [0]:
workflow = StateGraph(SimpleState)
workflow.add_node("addition", addition)
workflow.add_node("subtraction", subtraction)

workflow.add_edge(START, "addition")
workflow.add_edge("addition", "subtraction")
workflow.add_edge("subtraction", END)
app = workflow.compile()

In [0]:
display(Image(app.get_graph().draw_mermaid_png()))

In [0]:
input = {"a": 1, "b": 2}

result = app.invoke(input)
type(result)

### A more iteresting example using reflection.  

 - Ask the model to write a poem.
 - Then, ask the model to critique the poem.
 - Pass the critique back to the model, but as a HumanMessage
 - Ask the model to update the poem
 - Continue for three rewrites

In [0]:
display(Image(app.get_graph().draw_mermaid_png()))

In [0]:
from langgraph.graph.message import add_messages

model = ChatDatabricks(endpoint="databricks-meta-llama-3-1-405b-instruct")
#model = ChatDatabricks(endpoint="databricks-mixtral-8x7b-instruct")

reflection_instruction = """You are a talented poet. Given a poem, provide detailed recommentations no how the poem could be improved. 

Poem:

{poem}
"""

writing_instruction = """You are a talented poet. A user has asked you to write a poem. Write the best poem you can based on the user's request. If the user provides a critique of your writing, revise your poem accordingly. Don't revise the poem unless the user provides feedback. If you receive feedback, return only the revised poem. Don't reference the feedback or thank the user for the feedback."""


critique_prompt = PromptTemplate.from_template(reflection_instruction)
 
writing_prompt = ChatPromptTemplate.from_messages(
  [
    ("system", writing_instruction),
    MessagesPlaceholder(variable_name="messages")
  ]
)


class ReflectionState(TypedDict):
  messages: Annotated[list, add_messages]


def writing_node(state: ReflectionState) -> Dict[str, Any]:

  writing_chain = writing_prompt | model

  poem_generation = writing_chain.invoke({"messages": state['messages']})

  return {"messages": poem_generation}


def critique_node(state: ReflectionState) -> Dict[str, Any]:

  poem = state['messages'][-1].content

  reflection_chain = critique_prompt | model
  critique = reflection_chain.invoke({"poem": poem})

  return {"messages": [HumanMessage(content=critique.content)]}


def continue_critiquing(state: ReflectionState) -> str:
  # Allow for three revisions
  if len(state['messages']) > 7:
    return END
  return "critique_node"


workflow = StateGraph(ReflectionState)
workflow.add_node("writing_node", writing_node)
workflow.add_node("critique_node", critique_node)

workflow.add_edge(START, "writing_node")
workflow.add_conditional_edges("writing_node", continue_critiquing)
workflow.add_edge("critique_node", "writing_node")
app = workflow.compile()

display(Image(app.get_graph().draw_mermaid_png()))

In [0]:
response = app.invoke({"messages": 
  [
  HumanMessage(content="Write a poem about a data scientist that loves working on analytical problems")
    ]
  }
)

In [0]:
for resp in response['messages']:
  print(f"{resp.type}: {resp.content}\n\n")

First poem

In [0]:
print(response['messages'][1].content)

Last poem

In [0]:
print(response['messages'][-1].content)